# About

The purpose of this document is to do the following:
- describe a set of data pulled from a Dataverse
- provide Python code and a simple computing environment for working with the data (without the need to manually build your own)
- perform a set of checks and validation on the data

**NOTE: the code and this environment may become deprecated at some point in time and may no longer function as originally designed. For context, this notebook was last run on:**

In [1]:
from datetime import datetime
currentDateAndTime = datetime.now()
print(currentDateAndTime)

2023-04-27 18:18:09.670567


## Using this Jupyter notebook on data from the Dataverse

This Jupyter notebook uses a Python environment, along with the Python `pandas` module, to retrieve STATA data from a Dataverse and copy it to a dataframe. Our Python environment is defined in the `environment.yml` configuration file, so if we needed some additional Python modules to build charts or graphs later, this is where we would add those modules. We chose STATA as the data source because the pandas module will automatically assign columns/variables as categorical if variable value labels are assigned in STATA. If we only downloaded the default .tab file from the Dataverse you would only have the raw data, which would require manually mapping the variable categories to the data.

Note: If you are more familiar with using the R project to work data than Python, then you could modify this notebook to use the 'foreign' package for R [https://cran.r-project.org/web/packages/foreign/foreign.pdf], but we are not including every possible use-case in this document and are only using Python.

## Retrieve data from the Dataverse

Now we can move on to the fun stuff. Below we will begin with some code we will need to work with our data. We need to specific the files we want to analyze from the Dataverse and import the Python module (pandas) that will help us describe and make sense of the data. No output will be printed from this code as we are simply retrieving the data.

In [1]:
# pull the original STATA file at a specific version (using ?format=original&version=5.2) from the Dataverse, otherwise we would simply be pulling the CSV file without the categories included
DATA_FILE1 = 'https://demo.dataverse.org/api/access/datafile/2062199?format=original&version=5.2'  # this first file contains some demographics; we want to be explicit on the data version and file format so everyone knows exactly which files are being used

DATA_FILE1SAS = 'https://demo.dataverse.org/api/access/datafile/2062440?format=original&version=6.1' # this is a cleaner version of 

# see https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html on how to group and describe data
import pandas as pd  # tell Python we want to use the pandas module
df1 = pd.read_stata(DATA_FILE1)  # read the data file from the URL defined in the `DATA_FILE1` variable and save it to a dataframe named `df1`
df1sas = pd.read_sas(DATA_FILE1SAS, format='sas7bdat')

## Print general stats from the STATA data

Now we will print some general statistics on the data we just downloaded and saved to dataframes.

In [14]:
print("Start dataset 1 statistics\n=======================================")
print(df1.head(3))  # print the first three rows of the dataframe, including the header (which might be wrapped to a second line)
# print(df1['E03RADRPAKKL'].dtype) # this should show the data type as categorical
# print(df1[['E03RADRPAKKL','E03RADRPAKKL','E03AGE','E03GENDER']].describe())  # prints general stats for select variables
print(df1.describe())  # prints general stats for all vars (this is useful because it can easily point out data errors, such as duplicates in a unique field where the `count` may be reported as 628 but the `unique` may be showing 627)
print("=======================================\nend dataset 1 statistics")

Start dataset 1 statistics
                           E03SUBJECTID E03RADLPAKKL  E03RADRPAKKL E03AGE   
0  e731b53a-c413-41bd-8018-14f3fb7cfb7b     Moderate      Moderate  50-54  \
1  86988f07-1c1d-42fc-b23b-6ef7b829a7e1          NaN          Mild  60-64   
2  3b58c0d3-a33b-4a0c-b231-34985f7d23eb         Mild  Questionable  65-70   

  E03GENDER  E03PASKL  E03PASKR  
0    Female      None  Moderate  
1    Female      None    Severe  
2    Female  Moderate  Moderate  
                                E03SUBJECTID E03RADLPAKKL  E03RADRPAKKL   
count                                    628          618           616  \
unique                                   627            6             6   
top     84882a65-ba24-42b2-bb29-0bd3ac001927        No OA  Questionable   
freq                                       2          237           216   

       E03AGE E03GENDER E03PASKL E03PASKR  
count     628       628      627      627  
unique      7         2        4        4  
top     65-70    Fem

## Data curation and performing automated data validation and checks

It can be difficult to spot data issues unless you know what to look for. As a data curator, it is important to define the values you expect, or do not expect to see in a data column. For example, our `E03SUBJECTID` values should all be unique in our `df1` dataset, meaning the total number of values should equal the total number of unique values.

In the stats above, if were had read the data carfully we might have spotted that our `E03SUBJECTID` statistics do not look correct with regard to the unique value count in the column. With that in mind we should create reproducible checks on our data to ensure the data meets criteria we define for our data. This is especially useful if you will be uploading new versions of the data to the Dataverse frequently since you do not want to manually check for the same issues on new data. Plus, having the validation defined in a file gives the end-user confidence that you are not simply manually checking your data (or not checking your data), and instead can verify the data meets specific conditions automatically themselves.

Since we expect our `E03SUBJECTID` column to contain unique values, we create a simple check using the Python code below:

In [15]:
# let us print our counts just so we have a visual on what the data looks like
print("total number of values for E03SUBJECTID: ",df1["E03SUBJECTID"].count())
print("total number of distinct values for E03SUBJECTID: ",df1["E03SUBJECTID"].unique().size)

# we post an error if the E03SUBJECTID column does not contain distinct values
if df1["E03SUBJECTID"].count()!=df1["E03SUBJECTID"].unique().size:
    raise RuntimeError("***ERROR: The E03SUBJECTID column contains a duplicate value***")
    
#df1.groupby(["E03GENDER"])["E03RADLPAKKL"].count()

# show some combined data (https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html)

total number of values for E03SUBJECTID:  628
total number of distinct values for E03SUBJECTID:  627


RuntimeError: ***ERROR: The E03SUBJECTID column contains a duplicate value***

As designed, the code throws an error saying that our `E03SUBJECTID` column contains a duplicate value. Now that we know our data does not pass inspection we can go back and correct the data and then rerun this notebook to verify the data passes inspection.

## Comparing STATA and SAS data

Now we will print statistics on the SAS data we just downloaded and saved to a dataframe. This SAS data file does not contain 
the duplicate values we had in our test STATA data file.

Spoiler alert, pandas heavily favors the STATA file format and as such does not support SAS formats (variable labels), thus we will only see the raw data and no variable labels. With that said, it makes it much more difficult to justify working with the SAS data within this notebook. Also, you will see that the stats are using float values, which we do not want. So stick with STATA for simplicity.

In [3]:
print("Start dataset 1 SAS statistics\n=======================================")
print(df1sas.head(3))  # print the first three rows of the dataframe, including the header (which might be wrapped to a second line)
# print(df1sas['E03RADRPAKKL'].dtype) # this should show the data type as categorical
# print(df1sas[['E03RADRPAKKL','E03RADRPAKKL','E03AGE','E03GENDER']].describe())  # prints general stats for select variables
print(df1sas.describe())  # prints general stats for all vars (this is useful because it can easily point out data errors, such as duplicates in a unique field where the `count` may be reported as 628 but the `unique` may be showing 627)
print("=======================================\nend dataset 1 SAS statistics")

# let us print our counts just so we have a visual on what the data looks like
print("total number of values for E03SUBJECTID: ",df1sas["E03SUBJECTID"].count())
print("total number of distinct values for E03SUBJECTID: ",df1sas["E03SUBJECTID"].unique().size)

# we post an error if the E03SUBJECTID column does not contain distinct values
if df1sas["E03SUBJECTID"].count()!=df1sas["E03SUBJECTID"].unique().size:
    raise RuntimeError("***ERROR: The SAS E03SUBJECTID column contains a duplicate value***")
    
#df1sas.groupby(["E03GENDER"])["E03RADLPAKKL"].count()

# show some combined data (https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html)

Start dataset 1 SAS statistics
                              E03SUBJECTID  E03RADLPAKKL  E03RADRPAKKL   
0  b'0053be32-a7d2-4216-84be-e725308c3569'           0.0           1.0  \
1  b'0060b8e1-97a8-4c46-91ed-ea20291e50b9'           3.0           4.0   
2  b'00b245a4-4b03-410d-8219-6d37a67817d1'           0.0           1.0   

   E03AGE  E03GENDER  E03PASKL  E03PASKR  
0     4.0        1.0       0.0       0.0  
1     6.0        2.0       2.0       0.0  
2     4.0        2.0       0.0       0.0  
       E03RADLPAKKL  E03RADRPAKKL      E03AGE   E03GENDER    E03PASKL   
count    627.000000    627.000000  627.000000  627.000000  627.000000  \
mean       1.261563      1.314195    4.580542    1.661882    0.846890   
std        4.094160      4.097670    1.859559    0.473447    1.049343   
min       -1.000000     -1.000000    1.000000    1.000000   -1.000000   
25%        0.000000      0.000000    3.000000    1.000000    0.000000   
50%        1.000000      1.000000    5.000000    2.000000    0

## Joining data

Now we will pull in another data file that we plan to join to the data we were just working with. This is another STATA file (as we will stop working with SAS data going forward). This data will contain duplicate `E03SUBJECTID` values unlike our first data file, however the `E03USIMGFN` values should be unique. As we did prior, we will confirm that `E03USIMGFN` does contain unique values.

In [9]:
DATA_FILE2 = 'https://demo.dataverse.org/api/access/datafile/2063000?format=original&version=7.0' 
df2 = pd.read_stata(DATA_FILE2)

print("Start dataset 2 STATA statistics\n=======================================")
print(df2.head(3))  # print the first three rows of the dataframe, including the header (which might be wrapped to a second line)
print(df2.describe())  # prints general stats for all vars (this is useful because it can easily point out data errors, such as duplicates in a unique field where the `count` may be reported as 628 but the `unique` may be showing 627)
print("=======================================\nend dataset 2 STATA statistics")

# let us print our counts just so we have a visual on what the data looks like
print("total number of values for E03USIMGFN: ",df2["E03USIMGFN"].count())
print("total number of distinct values for E03USIMGFN: ",df2["E03USIMGFN"].unique().size)

# we post an error if the E03USIMGFN column does not contain distinct values
if df2["E03USIMGFN"].count()!=df2["E03USIMGFN"].unique().size:
    raise RuntimeError("***ERROR: The E03USIMGFN column contains a duplicate value***")

Start dataset 2 STATA statistics
                           E03SUBJECTID   
0  e731b53a-c413-41bd-8018-14f3fb7cfb7b  \
1  e731b53a-c413-41bd-8018-14f3fb7cfb7b   
2  e731b53a-c413-41bd-8018-14f3fb7cfb7b   

                                 E03USIMGFN         E03USIMGBP E03BODYSIDE  
0  049c9252-6b05-4746-b6b4-d1b5172728a9.png      SUPRAPAT LONG  Right side  
1  ac2201c4-a17b-414d-ac54-52a4724da98f.png      SUPRAPAT LONG   Left side  
2  41914520-7091-4f29-9c09-270d3ab01144.png  SUPRAPAT LONG CPD  Right side  
                                E03SUBJECTID   
count                                   8664  \
unique                                   632   
top     aa901e5d-1ab4-4db1-adec-b2d88a3e815f   
freq                                      22   

                                      E03USIMGFN     E03USIMGBP E03BODYSIDE  
count                                       8664           8664        8664  
unique                                      8664              7           2  
top     049

## What next

At this point we have a problem. The number of records in our second data file is 64 more than what we have in our images archive. We only have 8600 images.

In [1]:
# we need to compare the list of image file references in df2 and compare to the contents of the zip archive
print("compare the list of image file references in df2 and compare to the contents of the zip archive")

# we will ignore this since it is not useful
# dfjoin = df2.join(df1, lsuffix='_df1', rsuffix='_df2')  # for information on joins visit https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html
# print(dfjoin)

# print("Start dataset join statistics\n=======================================")
# print(dfjoin.head(3))  # print the first three rows of the dataframe, including the header (which might be wrapped to a second line)
# print(dfjoin.describe())  # prints general stats for all vars (this is useful because it can easily point out data errors, such as duplicates in a unique field where the `count` may be reported as 628 but the `unique` may be showing 627)
# print("=======================================\nend dataset join statistics")

compare the list of image file references in df2 and compare to the contents of the zip archive
